Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

subNumber = 8

In [8]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### Define Pipeline Components

In [118]:
from xgboost import XGBClassifier

randomforest_pipeline = Pipeline([('vect', TfidfVectorizer(stop_words='english', ngram_range=(1,4))),
                 ('randomforest', RandomForestClassifier())])

xgboost_pipeline = Pipeline([('vect', TfidfVectorizer(stop_words='english', ngram_range=(1,4))),
                 ('model', XGBClassifier(seed=1337))])

randomforest_pipeline_hardcoded = Pipeline([('vect', TfidfVectorizer(stop_words='english', ngram_range=(1,4),
                                                                     max_df=1.0, min_df=.02, max_features=750)),
                 ('randomforest', RandomForestClassifier(max_depth=20, n_estimators=100))])

xgboost_pipeline_hardcoded = Pipeline([('vect', TfidfVectorizer(stop_words='english', ngram_range=(1,4), max_df=0.5,
                                                                max_features=775, min_df=0.01)),
                 ('model', XGBClassifier(seed=1337, colsample_bytree=0.28, gamma=2.75, learning_rate=0.22,
                                         max_depth=60, min_child_leaf=7, min_child_weight=1, n_estimators=275,
                                         scale_pos_weight=100, subsample=0.8))])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [124]:
from skopt import BayesSearchCV
import warnings
warnings.filterwarnings('ignore') #ignore warnings generated by bayes search

tune = True

if tune:
    parameters = {
        'vect__max_df': ( 0.75, 1.0),
        'vect__min_df': (.02, .03),
        'vect__max_features': (700, 750, 800),
        'randomforest__n_estimators':(100, 125, 150),
        'randomforest__max_depth':(20, 25, 30)
    }

    xgboost_distributions = {
        'vect__max_df': (0.5, 1.0),
        'vect__min_df': (.01, .02, .03, .04, .05, .10),
        'vect__max_features': (725, 750, 775),
        'model__n_estimators': [250, 275, 300, 325, 350],
        'model__max_depth': [50, 60, 70, 75, 80, 85, 95, 100],
        'model__learning_rate': [0.22, 0.23, 0.24, 0.25, 0.26],
        'model__min_child_leaf':[6, 7, 8],
        'model__min_child_weight': [1, 2, 3, 5, 7, 9, 12, 15],
        'model__colsample_bytree':[0.28, 0.30, 0.32],
        'model__subsample':[0.8, 0.9, 1],
        'model__gamma':[2.5, 2.75, 3, 3.25, 3.50],
        'model__scale_pos_weight': [100, 110, 120, 130, 140, 150]
     }

    randomforest_pipeline = Pipeline([('vect', TfidfVectorizer(stop_words='english', ngram_range=(1,4))),
                 ('randomforest', RandomForestClassifier())])

    xgboost_pipeline = Pipeline([('vect', TfidfVectorizer(stop_words='english', ngram_range=(1,4))),
                 ('model', XGBClassifier(seed=1337))])

    def report_step_xgboost(optim_result):
        global previous_score
        global i
        global total_i
        score = xgboost_search.best_score_
        print("XGboost best score: %s" % score)
        if score > previous_score:
            i = 0
        else:
            i += 1
        print("Iterations since improvement: ", i)
        total_i += 1
        print("Total iterations: ", total_i)
        if i > 5:
            print("Over 5 iterations with no improvement, aborting...")
            return True
        previous_score = score
        print("--------------------------------------")

    xgboost_search = BayesSearchCV(
        xgboost_pipeline,
        search_spaces=xgboost_distributions,
        n_iter=1000,
        cv=5,
        scoring='accuracy',
        random_state=1337,
        n_points=5,
        n_jobs=15
    )

    i = 0
    total_i = 0
    previous_score = 0

    xgboost_search.fit(train['description'], train['ratingCategory'], callback=report_step_xgboost)
    xgboost_train_pred = xgboost_search.predict(train['description'])
    xgboost_test_pred = xgboost_search.predict(test['description'])



# parameters = {
#     'vect__max_df': (0.75, 1.0),
#     'clf__max_depth':(5,10,15,20)
# }

# grid_search = GridSearchCV(randomforest_pipeline,parameters, cv=5, n_jobs=15, verbose=3)
# grid_search.fit(train['description'], train['ratingCategory'])

XGboost best score: 0.7369708832884757
Iterations since improvement:  0
Total iterations:  1
--------------------------------------
XGboost best score: 0.7369708832884757
Iterations since improvement:  1
Total iterations:  2
--------------------------------------
XGboost best score: 0.7418644482505505
Iterations since improvement:  0
Total iterations:  3
--------------------------------------
XGboost best score: 0.7418644482505505
Iterations since improvement:  1
Total iterations:  4
--------------------------------------
XGboost best score: 0.7418644482505505
Iterations since improvement:  2
Total iterations:  5
--------------------------------------
XGboost best score: 0.7418644482505505
Iterations since improvement:  3
Total iterations:  6
--------------------------------------
XGboost best score: 0.7418644482505505
Iterations since improvement:  4
Total iterations:  7
--------------------------------------
XGboost best score: 0.7418644482505505
Iterations since improvement:  5
Tota

In [125]:
xgboost_search.best_params_

OrderedDict([('model__colsample_bytree', 0.28),
             ('model__gamma', 2.75),
             ('model__learning_rate', 0.22),
             ('model__max_depth', 60),
             ('model__min_child_leaf', 7),
             ('model__min_child_weight', 1),
             ('model__n_estimators', 275),
             ('model__scale_pos_weight', 100),
             ('model__subsample', 0.8),
             ('vect__max_df', 0.5),
             ('vect__max_features', 775),
             ('vect__min_df', 0.01)])

In [126]:
xgboost_search.best_score_

0.7418644482505505

In [135]:
randomforest_pipeline_hardcoded.fit(train['description'], train['ratingCategory'])

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=750,
                                 min_df=0.02, ngram_range=(1, 4), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pat...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=20, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,


In [136]:
from sklearn.model_selection import cross_val_score
import numpy as np

print(np.mean(cross_val_score(randomforest_pipeline_hardcoded, train['description'], train['ratingCategory'], cv=5,
                                                n_jobs=14)))

0.7230283732302268


### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [132]:
# Predictions on test sample
# pred = xgboost_search.predict(test['description'])
pred = xgboost_test_pred

In [133]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [134]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,0
3,3764,1
4,2306,1


In [130]:
subNumber += 1

In [131]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [9]:
# Apply to your Dataset

from sklearn.ensemble import GradientBoostingClassifier
import spacy
nlp = spacy.load("en_core_web_lg")

def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]



from scipy.stats import randint

In [37]:
X_train = get_word_vectors(train['description'])
X_test = get_word_vectors(test['description'])

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [79]:
from skopt import BayesSearchCV
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_selection import SelectKBest
import warnings
warnings.filterwarnings('ignore') #ignore warnings generated by bayes search

tune = False
xgboost_tune = False
forest_tune = False

if tune:
    parameters = {
        # 'vect__max_df': ( 0.75, 0.85, 0.95, 1.0),
        # 'vect__min_df': (.02, .03, .05, .07),
        'max_features': (225, 250, 275, 300),
        'n_estimators':(115, 125, 140),
        'max_depth':(215, 225, 240),
        'min_samples_leaf': [8, 9, 10, 11, 12, 13]
    }

    xgboost_distributions = {
        # 'vect__max_df': (0.5, 1.0),
        # 'vect__min_df': (.01, .02, .03, .04, .05, .10),
        # 'vect__max_features': (725, 750, 775),
        'n_estimators': [390, 400, 410],
        'max_depth': [48, 50, 52],
        'learning_rate': [0.07, 0.08, 0.09],
        'min_child_leaf':[5, 6, 7],
        'min_child_weight': [24, 25, 26],
        'colsample_bytree':[0.09, 0.10, 0.11],
        'subsample':[0.88, 0.9, 0.92],
        'gamma':[3.75, 4, 4.25],
        'scale_pos_weight': [125, 130, 135]
     }

    randomforest = RandomForestClassifier(random_state=1337)
    xgboost = XGBClassifier(seed=1337)

    def report_step_forest(optim_result):
        global previous_score
        global i
        global total_i
        score = forest_search.best_score_
        print("Random Forest best score: %s" % score)
        if score > previous_score:
            i = 0
        else:
            i += 1
        print("Iterations since improvement: ", i)
        total_i += 1
        print("Total iterations: ", total_i)
        if i > 2:
            print("Over 2 iterations with no improvement, aborting...")
            return True
        previous_score = score

    def report_step_xgboost(optim_result):
        global previous_score
        global i
        global total_i
        score = xgboost_search.best_score_
        best_params = xgboost_search.best_params_
        print("XGboost best score: %s" % score)
        print("XGboost best params: %s" % best_params)
        if score > previous_score:
            i = 0
        else:
            i += 1
        print("Iterations since improvement: ", i)
        total_i += 1
        print("Total iterations: ", total_i)
        if i > 3:
            print("Over 3 iterations with no improvement, aborting...")
            return True
        previous_score = score
        print("--------------------------------------")

    xgboost_search = BayesSearchCV(
        xgboost,
        search_spaces=xgboost_distributions,
        n_iter=1000,
        cv=5,
        scoring='accuracy',
        random_state=1337,
        n_points=5,
        n_jobs=15
    )

    forest_search = BayesSearchCV(
        randomforest,
        search_spaces=parameters,
        n_iter=1000,
        cv=5,
        scoring='accuracy',
        random_state=1337,
        n_points=5,
        n_jobs=15
    )


    if forest_tune:
        i = 0
        total_i = 0
        previous_score = 0

        forest_search.fit(X_train, train['ratingCategory'], callback=report_step_forest)
        forest_train_pred = forest_search.predict(X_train)
        forest_test_pred = forest_search.predict(X_test)

    if xgboost_tune:
        i = 0
        total_i = 0
        previous_score = 0

        xgboost_search.fit(X_train, train['ratingCategory'], callback=report_step_xgboost)
        xgboost_train_pred = xgboost_search.predict(X_train)
        xgboost_test_pred = xgboost_search.predict(X_test)

else:

    randomforest = RandomForestClassifier(random_state=1337, max_depth=225, max_features=250,
                                                   min_samples_leaf=13, n_estimators=125, n_jobs=15)
    xgboost = XGBClassifier(seed=1337, colsample_bytree=0.1, gamma=4, learning_rate=0.08, max_depth=50,
                            min_child_leaf=6, min_child_weight=25, n_estimators=400, scale_pos_weight=130,
                            subsample=0.9, n_jobs=15)

    logistic = LogisticRegressionCV(max_iter=300, n_jobs=15)

    randomforest.fit(X_train, train['ratingCategory'])
    forest_train_pred = randomforest.predict(X_train)
    forest_test_pred = randomforest.predict(X_test)

    xgboost.fit(X_train, train['ratingCategory'])
    xgboost_train_pred = xgboost.predict(X_train)
    xgboost_test_pred = xgboost.predict(X_test)

    logistic.fit(X_train, train['ratingCategory'])
    logistic_train_pred = logistic.predict(X_train)
    logistic_test_pred = logistic.predict(X_test)



[21:22:14] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { min_child_leaf, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [38]:


# xgboost_test = xgboost.fit(X_train, train['ratingCategory'])
# randomforest_test = randomforest_pipeline.fit(X_train, train['ratingCategory'])

In [80]:
from sklearn.model_selection import cross_val_score

if tune:
    if logistic_tune:
        print("Logistic regression best score:", logistic_search.best_score_)
        print("Logistic regression best params:", logistic_search.best_params_)
else:
    print("XGboost cross val score:", np.mean(cross_val_score(xgboost, X_train, train['ratingCategory'], cv=5,
                                                n_jobs=14)))

    print("Random forest cross val score:", np.mean(cross_val_score(randomforest, X_train, train['ratingCategory'], cv=5,
                                                n_jobs=14)))

    print("Logistic regression cross val score:", np.mean(cross_val_score(logistic, X_train, train['ratingCategory'], cv=5,
                                            n_jobs=14)))

XGboost cross val score: 0.7487175036585036
Random forest cross val score: 0.7386849736497952
Logistic regression cross val score: 0.7514135740214811


In [43]:
xgboost_search.best_params_

OrderedDict([('colsample_bytree', 0.1),
             ('gamma', 4),
             ('learning_rate', 0.08),
             ('max_depth', 50),
             ('min_child_leaf', 6),
             ('min_child_weight', 25),
             ('n_estimators', 400),
             ('scale_pos_weight', 130),
             ('subsample', 0.9)])

In [44]:
xgboost_search.best_score_

0.7487154391974553

In [73]:
from sklearn.linear_model import LogisticRegressionCV

model = LogisticRegressionCV(max_iter=300, n_jobs=15)
model.fit(X_train, train['ratingCategory'])

logistic_test = model.predict(X_test)

In [72]:
print("Logistic regression cross val score:", np.mean(cross_val_score(model, X_train, train['ratingCategory'], cv=5,
                                            n_jobs=14)))


Logistic regression cross val score: 0.7514135740214811


In [75]:
# import pandas as pd
#
# df = pd.DataFrame(X_tfidf.todense(), columns=vicktor.get_feature_names())

In [76]:
# rfc = RandomForestClassifier()
# rfc.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [77]:
# rfc.score(X,train['ratingCategory'])

1.0

In [88]:
# len(X[0])

300

### Make a Submission File

In [74]:
# Predictions on test sample
pred = logistic_test

In [75]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [76]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [77]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?